In [1]:
##THE HYBRID WITH SLIME AND IALS RESULTS ARE BETTER THAN ANY OTHER ALGORITHM TESTED BEFORE
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores, MergeTwoModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [3]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)
stacked_train = combine_matrices(ICM=ICM, URM=URM_train)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13639 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13625 ( 0.2%) Users that have less than 1 test interactions


In [7]:
from skopt.space import Integer
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
hyperparameters_range_dictionary = {
    "num_factors": Integer(10,80),
    "topK": Integer(10,1000)
}
earlystopping_keywargs = {"validation_every_n": 5,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 4,
                          "validation_metric": "MAP",
                          }

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

tuning_class = SearchBayesianSkopt(recommender_class=PureSVDItemRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 70
n_random_starts = n_cases*0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize= 10,
                    n_cases= n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=PureSVDItemRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'num_factors': 28, 'topK': 42}
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 13625 (100.0%) in 12.02 sec. Users per second: 1133
SearchBayesianSkopt: New best config found. Config 0: {'num_factors': 28, 'topK': 42} - results: PRECISION: 0.1963156, PRECISION_RECALL_MIN_DEN: 0.1991296, RECALL: 0.0549914, MAP: 0.0931266, MAP_MIN_DEN: 0.0944716, MRR: 0.4214206, NDCG: 0.2070022, F1: 0.0859162, HIT_RATE: 0.8623853, ARHR_ALL_HITS: 0.6253175, NOVELTY: 0.0054152, AVERAGE_POPULARITY: 0.5982497, DIVERSITY_MEAN_INTER_LIST: 0.8363351, DIVERSITY_HERFINDAHL: 0.9836274, COVERAGE_ITEM: 0.0337228, COVERAGE_ITEM_CORRECT: 0.0204330, COVERAGE_USER: 0.9981685, COVERAGE_USER_CORRECT: 0.8608059, DIVERSITY_GINI: 0.0047193, SHANNON_ENTROPY: 6.7398410, RATIO_DIVERSITY_HERFINDAHL: 0.9840070, RATIO_DIVERSITY_GINI: 0.019

PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 13625 (100.0%) in 27.87 sec. Users per second: 489
SearchBayesianSkopt: Config 5 is suboptimal. Config: {'num_factors': 68, 'topK': 542} - results: PRECISION: 0.1847780, PRECISION_RECALL_MIN_DEN: 0.1871378, RECALL: 0.0502605, MAP: 0.0845556, MAP_MIN_DEN: 0.0855807, MRR: 0.3947686, NDCG: 0.1923982, F1: 0.0790256, HIT_RATE: 0.8372844, ARHR_ALL_HITS: 0.5763239, NOVELTY: 0.0052976, AVERAGE_POPULARITY: 0.6469631, DIVERSITY_MEAN_INTER_LIST: 0.8244048, DIVERSITY_HERFINDAHL: 0.9824344, COVERAGE_ITEM: 0.0300127, COVERAGE_ITEM_CORRECT: 0.0191594, COVERAGE_USER: 0.9981685, COVERAGE_USER_CORRECT: 0.8357509, DIVERSITY_GINI: 0.0035536, SHANNON_ENTROPY: 6.3665979, RATIO_DIVERSITY_HERFINDAHL: 0.9828136, RATIO_DIVERSITY_GINI: 0.0143447, RATIO_SHANNON_ENTROPY: 0.5138816, RATIO_AVERAGE_POPULARITY: 3.1639517, RATIO_NOVELTY: 0.0276007, 

Iteration No: 6 ended. Eval

PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 13625 (100.0%) in 30.17 sec. Users per second: 452
SearchBayesianSkopt: Config 12 is suboptimal. Config: {'num_factors': 66, 'topK': 711} - results: PRECISION: 0.1850569, PRECISION_RECALL_MIN_DEN: 0.1876103, RECALL: 0.0504373, MAP: 0.0848283, MAP_MIN_DEN: 0.0859433, MRR: 0.3960551, NDCG: 0.1929606, F1: 0.0792696, HIT_RATE: 0.8397798, ARHR_ALL_HITS: 0.5781472, NOVELTY: 0.0053037, AVERAGE_POPULARITY: 0.6428378, DIVERSITY_MEAN_INTER_LIST: 0.8309681, DIVERSITY_HERFINDAHL: 0.9830907, COVERAGE_ITEM: 0.0301235, COVERAGE_ITEM_CORRECT: 0.0194917, COVERAGE_USER: 0.9981685, COVERAGE_USER_CORRECT: 0.8382418, DIVERSITY_GINI: 0.0036774, SHANNON_ENTROPY: 6.4209200, RATIO_DIVERSITY_HERFINDAHL: 0.9834701, RATIO_DIVERSITY_GINI: 0.0148444, RATIO_SHANNON_ENTROPY: 0.5182663, RATIO_AVERAGE_POPULARITY: 3.1437771, RATIO_NOVELTY: 0.0276327, 

Iteration No: 13 ended. Ev

PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 13625 (100.0%) in 31.36 sec. Users per second: 434
SearchBayesianSkopt: Config 18 is suboptimal. Config: {'num_factors': 48, 'topK': 827} - results: PRECISION: 0.1993028, PRECISION_RECALL_MIN_DEN: 0.2020004, RECALL: 0.0554473, MAP: 0.0934858, MAP_MIN_DEN: 0.0946906, MRR: 0.4185632, NDCG: 0.2079995, F1: 0.0867579, HIT_RATE: 0.8635596, ARHR_ALL_HITS: 0.6234998, NOVELTY: 0.0052712, AVERAGE_POPULARITY: 0.6767950, DIVERSITY_MEAN_INTER_LIST: 0.7951565, DIVERSITY_HERFINDAHL: 0.9795098, COVERAGE_ITEM: 0.0287391, COVERAGE_ITEM_CORRECT: 0.0201008, COVERAGE_USER: 0.9981685, COVERAGE_USER_CORRECT: 0.8619780, DIVERSITY_GINI: 0.0034983, SHANNON_ENTROPY: 6.3002152, RATIO_DIVERSITY_HERFINDAHL: 0.9798879, RATIO_DIVERSITY_GINI: 0.0141215, RATIO_SHANNON_ENTROPY: 0.5085235, RATIO_AVERAGE_POPULARITY: 3.3098434, RATIO_NOVELTY: 0.0274635, 

Iteration No: 19 ended. Ev

EvaluatorHoldout: Processed 13639 (100.0%) in 31.85 sec. Users per second: 428
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'num_factors': 25, 'topK': 934} - results:
CUTOFF: 10 - PRECISION: 0.2451426, PRECISION_RECALL_MIN_DEN: 0.2472982, RECALL: 0.0574394, MAP: 0.1235936, MAP_MIN_DEN: 0.1246726, MRR: 0.4815581, NDCG: 0.2553731, F1: 0.0930712, HIT_RATE: 0.9117237, ARHR_ALL_HITS: 0.7676624, NOVELTY: 0.0053792, AVERAGE_POPULARITY: 0.6140531, DIVERSITY_MEAN_INTER_LIST: 0.8550073, DIVERSITY_HERFINDAHL: 0.9854945, COVERAGE_ITEM: 0.0297359, COVERAGE_ITEM_CORRECT: 0.0238662, COVERAGE_USER: 0.9991941, COVERAGE_USER_CORRECT: 0.9109890, DIVERSITY_GINI: 0.0054787, SHANNON_ENTROPY: 6.9527131, RATIO_DIVERSITY_HERFINDAHL: 0.9858748, RATIO_DIVERSITY_GINI: 0.0221155, RATIO_SHANNON_ENTROPY: 0.5611901, RATIO_AVERAGE_POPULARITY: 3.0030063, RATIO_NOVELTY: 0.0280261, 


SearchBayesianSkopt: Saving model in logs/PureSVDItemRecommender

PureSVDItemRecommender: Saving model in file 'log

...

Iteration No: 37 ended. Search finished for the next optimal point.
Time taken: 44.4636
Function value obtained: -0.0946
Current minimum: -0.1010
Iteration No: 38 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'num_factors': 50, 'topK': 418}
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 13625 (100.0%) in 21.18 sec. Users per second: 643
SearchBayesianSkopt: Config 37 is suboptimal. Config: {'num_factors': 50, 'topK': 418} - results: PRECISION: 0.1982312, PRECISION_RECALL_MIN_DEN: 0.2010200, RECALL: 0.0548858, MAP: 0.0921595, MAP_MIN_DEN: 0.0933195, MRR: 0.4113103, NDCG: 0.2058926, F1: 0.0859688, HIT_RATE: 0.8600367, ARHR_ALL_HITS: 0.6145120, NOVELTY: 0.0052787, AVERAGE_POPULARITY: 0.6700757, DIVERSITY_MEAN_INTER_LIST: 0.8030711, DIVERSITY_HERFINDAHL: 0.9803012, COVERAGE_ITEM: 0.0300127, COVERAGE_ITEM_CORRECT: 0.0197685, COVERAGE_USER: 0.9981685, COVERA

Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 23.6978
Function value obtained: -0.0936
Current minimum: -0.1011
Iteration No: 44 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'num_factors': 80, 'topK': 191}
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 13625 (100.0%) in 17.41 sec. Users per second: 783
SearchBayesianSkopt: Config 43 is suboptimal. Config: {'num_factors': 80, 'topK': 191} - results: PRECISION: 0.1777908, PRECISION_RECALL_MIN_DEN: 0.1800247, RECALL: 0.0480682, MAP: 0.0809662, MAP_MIN_DEN: 0.0819995, MRR: 0.3875640, NDCG: 0.1858262, F1: 0.0756763, HIT_RATE: 0.8228991, ARHR_ALL_HITS: 0.5585787, NOVELTY: 0.0053567, AVERAGE_POPULARITY: 0.6016457, DIVERSITY_MEAN_INTER_LIST: 0.8652717, DIVERSITY_HERFINDAHL: 0.9865208, COVERAGE_ITEM: 0.0323384, COVERAGE_ITEM_CORRECT: 0.0198239, COVERAGE_USER: 0.9981685, COVERA

Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 30.5202
Function value obtained: -0.0952
Current minimum: -0.1011
Iteration No: 51 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'num_factors': 40, 'topK': 922}
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 13625 (100.0%) in 26.49 sec. Users per second: 514
SearchBayesianSkopt: Config 50 is suboptimal. Config: {'num_factors': 40, 'topK': 922} - results: PRECISION: 0.2066642, PRECISION_RECALL_MIN_DEN: 0.2094369, RECALL: 0.0575173, MAP: 0.0975813, MAP_MIN_DEN: 0.0988279, MRR: 0.4269187, NDCG: 0.2151704, F1: 0.0899894, HIT_RATE: 0.8746422, ARHR_ALL_HITS: 0.6439641, NOVELTY: 0.0053059, AVERAGE_POPULARITY: 0.6557355, DIVERSITY_MEAN_INTER_LIST: 0.8180118, DIVERSITY_HERFINDAHL: 0.9817952, COVERAGE_ITEM: 0.0308987, COVERAGE_ITEM_CORRECT: 0.0223157, COVERAGE_USER: 0.9981685, COVERA

Iteration No: 57 ended. Search finished for the next optimal point.
Time taken: 45.7116
Function value obtained: -0.0856
Current minimum: -0.1011
Iteration No: 58 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'num_factors': 25, 'topK': 505}
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 13625 (100.0%) in 21.76 sec. Users per second: 626
SearchBayesianSkopt: Config 57 is suboptimal. Config: {'num_factors': 25, 'topK': 505} - results: PRECISION: 0.2101064, PRECISION_RECALL_MIN_DEN: 0.2128631, RECALL: 0.0581587, MAP: 0.1006136, MAP_MIN_DEN: 0.1018593, MRR: 0.4335862, NDCG: 0.2193933, F1: 0.0911003, HIT_RATE: 0.8746422, ARHR_ALL_HITS: 0.6583548, NOVELTY: 0.0053745, AVERAGE_POPULARITY: 0.6177904, DIVERSITY_MEAN_INTER_LIST: 0.8488312, DIVERSITY_HERFINDAHL: 0.9848769, COVERAGE_ITEM: 0.0299574, COVERAGE_ITEM_CORRECT: 0.0228141, COVERAGE_USER: 0.9981685, COVERA

Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 46.5300
Function value obtained: -0.0999
Current minimum: -0.1011
Iteration No: 65 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'num_factors': 22, 'topK': 742}
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 13625 (100.0%) in 24.13 sec. Users per second: 565
SearchBayesianSkopt: Config 64 is suboptimal. Config: {'num_factors': 22, 'topK': 742} - results: PRECISION: 0.2100183, PRECISION_RECALL_MIN_DEN: 0.2127752, RECALL: 0.0580345, MAP: 0.1009369, MAP_MIN_DEN: 0.1021910, MRR: 0.4331741, NDCG: 0.2196237, F1: 0.0909396, HIT_RATE: 0.8731009, ARHR_ALL_HITS: 0.6597341, NOVELTY: 0.0053759, AVERAGE_POPULARITY: 0.6201631, DIVERSITY_MEAN_INTER_LIST: 0.8397453, DIVERSITY_HERFINDAHL: 0.9839684, COVERAGE_ITEM: 0.0290714, COVERAGE_ITEM_CORRECT: 0.0238662, COVERAGE_USER: 0.9981685, COVERA